<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-04 12:38:59
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: 8.99 K (0.06%)
Current PnL: -30.42 L (-19.4%)
CY Booked + Current PnL: -15.68 L (-10.0%)
-------------------
Total profit:  1.24 L
Total loss:  -31.66 L
-------------------
Total Booked + Current PnL: 11.30 L (8.7%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.6%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.86 L (68.93%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,1.04,6.69,11.75,19.23,18862.0,10064.0,160524.0,484.83,25.73,56.0,L-LC,12.64,182.0,0.53,1.15,29.39,XY25,ATH,MINING
50,MASFIN,-1.62,-4.49,27.28,21.57,25530.0,-4395.0,93585.0,397.04,-17.80,49.0,H-SC,3.45,168.0,-0.17,0.67,36.57,XR,ATH,FINANCE
77,TTKPRESTIG,-1.06,-27.97,38.95,0.09,28275.0,-28183.0,72594.0,770.00,68.41,33.0,M-SC,7.63,253.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
57,RECLTD,1.33,2.18,18.16,20.74,37700.0,4428.0,207598.0,446.00,35.83,58.0,M-MC,4.42,189.0,0.12,1.49,13.03,XY25,NTT,FINANCE
51,MEDANTA,1.42,-5.73,31.62,24.08,37841.0,-7276.0,119674.0,1486.00,-13.35,51.0,X-SC,10.64,91.0,-0.19,0.86,9.61,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SFL,16.17,-40.15,111.59,26.63,175116.0,-105290.0,156928.0,1287.00,18.85,65.0,M-SC,16.43,241.0,-0.60,1.13,20.46,XY24,NTT,MISC
23,DIXON,5.40,-19.39,62.83,31.26,116884.0,-44736.0,186032.0,18931.72,-62.32,55.0,X-MC,27.71,56.0,-0.38,1.34,14.17,X40N,ATH,IT
55,RAJESHEXPO,4.51,-65.78,192.41,0.07,90327.0,-90232.0,46945.0,518.00,1620.96,55.0,L-SC,12.03,269.0,-1.00,0.34,16.75,OX40N,NTT,JEWELLERY
5,ANGELONE,3.38,9.55,13.37,24.20,44709.0,29150.0,334400.0,3033.00,55.90,58.0,X-SC,1.47,99.0,0.65,2.41,40.05,X40N,NTT,FINANCE
65,SIEMENS,2.69,-11.39,41.64,25.51,68666.0,-21190.0,164905.0,4671.50,1.14,68.0,H-LC,3.80,50.0,-0.31,1.19,21.52,AR,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-8.00,3.71,29.44,34.24,95535.0,11618.0,324506.0,1972.00,-19.24,33.0,X-LC,6.85,6.0,0.12,2.33,12.30,X40,NTT,IT
73,TCS,-6.86,-17.40,43.52,18.55,135968.0,-65811.0,312426.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.25,6.29,X40,ATH,IT
33,HCLTECH,-5.33,4.16,18.89,23.83,47600.0,10061.0,251985.0,1908.19,8.94,38.0,X-LC,7.56,13.0,0.21,1.81,20.90,X40,ATH,IT
30,HAPPSTMNDS,-5.32,-50.49,273.35,84.85,173457.0,-64709.0,63456.0,1480.71,-41.05,32.0,H-SC,22.71,153.0,-0.37,0.46,0.00,AR,ATH,IT
84,WIPRO,-4.33,-4.77,80.89,72.27,130534.0,-8076.0,161372.0,420.00,-18.33,37.0,M-LC,2.54,101.0,-0.06,1.16,3.73,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.09,-6.33,121.95,107.91,168663.0,-9339.0,138305.0,1641.55,-23.79,48.0,M-SC,11.42,220.0,-0.06,0.99,4.05,OX40N,ATH,CABLES
66,SIS,1.58,-20.23,52.01,21.26,46067.0,-22458.0,88574.0,528.00,2094.82,64.0,H-SC,4.79,163.0,-0.49,0.64,19.67,OX40N,NTT,MISC
72,TATAELXSI,-1.13,-21.51,68.22,32.03,70590.0,-28361.0,103474.0,9161.00,-15.03,51.0,H-SC,6.47,157.0,-0.40,0.74,15.90,OX40N,NTT,IT
48,KANSAINER,-2.79,-25.98,53.31,13.47,106408.0,-70065.0,199602.0,340.00,-69.48,40.0,H-SC,5.21,159.0,-0.66,1.44,1.49,XY24,NTT,PAINTS
40,INDIGOPNTS,-0.18,-26.97,36.96,0.02,47114.0,-47087.0,127472.0,1408.00,94.20,30.0,M-SC,4.79,240.0,-1.00,0.92,10.88,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,1.33,2.18,18.16,20.74,37700.0,4428.0,207598.0,446.00,35.83,58.0,M-MC,4.42,189.0,0.12,1.49,13.03,XY25,NTT,FINANCE
18,COALINDIA,1.04,6.69,11.75,19.23,18862.0,10064.0,160524.0,484.83,25.73,56.0,L-LC,12.64,182.0,0.53,1.15,29.39,XY25,ATH,MINING
1,ABB,2.36,6.45,36.75,45.57,102347.0,16877.0,278496.0,7934.00,-31.59,72.0,H-MC,5.57,121.0,0.16,2.00,23.67,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,2.36,6.45,36.75,45.57,102347.0,16877.0,278496.0,7934.00,-31.59,72.0,H-MC,5.57,121.0,0.16,2.00,23.67,AR,NTT,ELECTRICAL
39,INDIAMART,-0.27,-2.82,112.73,106.72,135111.0,-3482.0,119854.0,4810.62,-51.96,56.0,H-SC,8.71,138.0,-0.03,0.86,19.90,AR,ATH,MISC
26,FINCABLES,0.09,-6.33,121.95,107.91,168663.0,-9339.0,138305.0,1641.55,-23.79,48.0,M-SC,11.42,220.0,-0.06,0.99,4.05,OX40N,ATH,CABLES
84,WIPRO,-4.33,-4.77,80.89,72.27,130534.0,-8076.0,161372.0,420.00,-18.33,37.0,M-LC,2.54,101.0,-0.06,1.16,3.73,XR,NTT,IT
85,ZYDUSLIFE,0.23,-4.57,42.13,35.63,84257.0,-9580.0,199994.0,1286.17,-18.07,54.0,H-MC,5.95,119.0,-0.11,1.44,11.34,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.64,-44.44,143.37,35.20,102487.0,-57186.0,71484.0,2096.00,-6.28,28.0,X-SC,20.67,93.0,-0.56,0.51,1.59,X40,NTT,FOOTWEAR
44,ITC,1.11,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,29.0,X-LC,18.51,1.0,-0.68,1.33,1.74,X40,NTT,FMCG
81,VBL,-1.86,-10.57,51.71,35.67,146051.0,-33399.0,282443.0,671.64,-21.24,29.0,X-LC,0.65,5.0,-0.23,2.03,1.81,X40N,ATH,FMCG
6,ASIANPAINT,0.88,-14.46,41.39,20.95,89138.0,-36406.0,215362.0,3460.26,-15.39,30.0,X-LC,9.88,36.0,-0.41,1.55,16.02,X40,ATH,PAINTS
42,INFY,-8.00,3.71,29.44,34.24,95535.0,11618.0,324506.0,1972.00,-19.24,33.0,X-LC,6.85,6.0,0.12,2.33,12.30,X40,NTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,-6.86,-17.40,43.52,18.55,135968.0,-65811.0,312426.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.25,6.29,X40,ATH,IT
81,VBL,-1.86,-10.57,51.71,35.67,146051.0,-33399.0,282443.0,671.64,-21.24,29.0,X-LC,0.65,5.0,-0.23,2.03,1.81,X40N,ATH,FMCG
5,ANGELONE,3.38,9.55,13.37,24.20,44709.0,29150.0,334400.0,3033.00,55.90,58.0,X-SC,1.47,99.0,0.65,2.41,40.05,X40N,NTT,FINANCE
34,HINDUNILVR,-0.18,-8.02,23.58,13.67,55753.0,-20623.0,236440.0,2922.00,-14.94,49.0,X-LC,2.32,9.0,-0.37,1.70,11.62,XY25,NTT,FMCG
21,DABUR,0.48,-1.68,46.15,43.70,112563.0,-4162.0,243906.0,735.00,-9.79,44.0,X-MC,2.57,73.0,-0.04,1.75,13.12,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.64,-44.44,143.37,35.20,102487.0,-57186.0,71484.0,2096.00,-6.28,28.0,X-SC,20.67,93.0,-0.56,0.51,1.59,X40,NTT,FOOTWEAR
44,ITC,1.11,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,29.0,X-LC,18.51,1.0,-0.68,1.33,1.74,X40,NTT,FMCG
81,VBL,-1.86,-10.57,51.71,35.67,146051.0,-33399.0,282443.0,671.64,-21.24,29.0,X-LC,0.65,5.0,-0.23,2.03,1.81,X40N,ATH,FMCG
53,PGHH,0.56,-10.11,49.28,34.19,94586.0,-21584.0,191936.0,17907.65,-36.80,39.0,X-MC,8.32,60.0,-0.23,1.38,2.06,X40,ATH,FMCG
3,ACC,0.78,-29.31,132.22,64.15,222394.0,-69751.0,168200.0,3906.00,-58.50,44.0,X-SC,7.69,84.0,-0.31,1.21,2.81,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.86,-28.16,98.83,42.84,46156.0,-18304.0,46702.0,424.00,-53.17,57.0,X-SC,21.48,83.0,-0.40,0.34,9.78,XY24,NTT,MISC
12,BATAINDIA,-0.64,-44.44,143.37,35.20,102487.0,-57186.0,71484.0,2096.00,-6.28,28.0,X-SC,20.67,93.0,-0.56,0.51,1.59,X40,NTT,FOOTWEAR
58,RELAXO,-2.06,-51.63,214.86,52.31,150870.0,-74942.0,70218.0,1176.00,-48.50,43.0,X-SC,12.03,92.0,-0.50,0.51,4.42,X40N,NTT,FOOTWEAR
51,MEDANTA,1.42,-5.73,31.62,24.08,37841.0,-7276.0,119674.0,1486.00,-13.35,51.0,X-SC,10.64,91.0,-0.19,0.86,9.61,XY24,NTT,HEALTHCARE
35,HONAUT,-1.86,-18.45,75.75,43.32,100611.0,-30052.0,132820.0,58357.33,-29.91,50.0,X-SC,8.95,90.0,-0.30,0.96,7.84,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.11,-23.14,44.13,10.78,81513.0,-55608.0,184710.0,452.00,-54.23,29.0,X-LC,18.51,1.0,-0.68,1.33,1.74,X40,NTT,FMCG
75,TMPV,-0.05,-23.79,61.36,22.98,105642.0,-53731.0,172167.0,600.00,-20.63,64.0,X-LC,4.44,2.0,-0.51,1.24,10.06,XY24,NTT,AUTO
73,TCS,-6.86,-17.40,43.52,18.55,135968.0,-65811.0,312426.0,4311.59,-28.80,38.0,X-LC,0.49,3.0,-0.48,2.25,6.29,X40,ATH,IT
81,VBL,-1.86,-10.57,51.71,35.67,146051.0,-33399.0,282443.0,671.64,-21.24,29.0,X-LC,0.65,5.0,-0.23,2.03,1.81,X40N,ATH,FMCG
42,INFY,-8.00,3.71,29.44,34.24,95535.0,11618.0,324506.0,1972.00,-19.24,33.0,X-LC,6.85,6.0,0.12,2.33,12.30,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.06,-31.94,95.27,32.89,50058.0,-24664.0,52543.0,1800.00,-748.59,59.0,L-MC,11.26,259.0,-0.49,0.38,44.70,XR,NTT,BANKS
67,SONACOMS,0.19,-8.10,51.36,39.09,47744.0,-8199.0,92960.0,804.02,-28.09,73.0,M-SC,1.87,227.0,-0.17,0.67,31.51,AR,ATH,AUTO
14,BSOFT,-3.59,-21.79,92.79,50.79,102075.0,-30646.0,110007.0,831.70,0.50,54.0,H-SC,6.40,171.0,-0.30,0.79,28.08,XR,ATH,IT
18,COALINDIA,1.04,6.69,11.75,19.23,18862.0,10064.0,160524.0,484.83,25.73,56.0,L-LC,12.64,182.0,0.53,1.15,29.39,XY25,ATH,MINING
65,SIEMENS,2.69,-11.39,41.64,25.51,68666.0,-21190.0,164905.0,4671.50,1.14,68.0,H-LC,3.80,50.0,-0.31,1.19,21.52,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,3.38,9.55,13.37,24.20,44709.0,29150.0,334400.0,3033.00,55.90,58.0,X-SC,1.47,99.0,0.65,2.41,40.05,X40N,NTT,FINANCE
1,ABB,2.36,6.45,36.75,45.57,102347.0,16877.0,278496.0,7934.00,-31.59,72.0,H-MC,5.57,121.0,0.16,2.00,23.67,AR,NTT,ELECTRICAL
67,SONACOMS,0.19,-8.10,51.36,39.09,47744.0,-8199.0,92960.0,804.02,-28.09,73.0,M-SC,1.87,227.0,-0.17,0.67,31.51,AR,ATH,AUTO
76,TRIDENT,2.33,-21.34,63.38,28.51,45994.0,-19686.0,72569.0,48.00,-6.81,69.0,M-SC,10.27,226.0,-0.43,0.52,24.18,XR,NTT,TEXTILES
62,SATIN,0.61,-12.52,73.56,51.83,177794.0,-34585.0,241699.0,274.00,-21.16,62.0,H-SC,3.04,148.0,-0.19,1.74,17.74,XY24,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.40
1,25,44.60
2,50,76.71


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.48
MC    30.79
LC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.64
X40      23.36
X40N     15.14
AR        9.08
XR        8.75
XY25      8.73
OX40N     7.42
SR        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.03
H-SC    22.99
X-LC    20.26
M-SC    11.31
X-SC    10.41
H-MC     4.89
M-MC     1.49
H-LC     1.19
M-LC     1.16
L-LC     1.15
L-SC     0.77
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.90,-14.25,52.87
FINANCE,13.55,-10.99,54.67
IT,12.40,-23.94,85.86
MISC,7.21,-29.76,82.85
ELECTRICAL,6.18,-9.26,49.13
PAINTS,5.30,-25.48,44.12
INSURANCE,4.79,-1.93,36.81
PHARMA,3.99,-5.89,39.23
AUTO,2.97,-27.14,71.12


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3268886.0,21
AR,1310663.0,10
XR,1274671.0,12
X40,1252430.0,15
X40N,1019062.0,10
OX40N,760979.0,10
XY25,405088.0,6
SR,293800.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3599841.0,24
X-MC,1689058.0,16
M-SC,1487070.0,15
X-LC,1075454.0,12
X-SC,846059.0,9
H-MC,398332.0,3
L-SC,183945.0,2
M-LC,130534.0,1
H-LC,68666.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1229389.0      6
           AR         934243.0      5
M-SC       XY24       837216.0      6
H-SC       XR         818632.0      7
X-MC       X40        571272.0      7
X-LC       X40        508119.0      6
X-MC       XY24       475883.0      3
           X40N       413429.0      4
X-SC       X40N       366629.0      4
M-SC       OX40N      346875.0      5
H-SC       OX40N      323777.0      4
X-SC       XY24       306391.0      3
H-SC       SR         293800.0      2
X-LC       X40N       239004.0      2
X-MC       XY25       228474.0      2
H-MC       XY24       211728.0      1
X-LC       XY24       208279.0      2
H-MC       AR         186604.0      2
M-SC       XR         181829.0      2
X-SC       X40        173039.0      2
M-LC       XR         130534.0      1
M-SC       AR         121150.0      2
X-LC       XY25       120052.0      2
L-SC       XR          93618.0      1
           OX40N       90327.0      1
H-LC       AR          68666.0      1
L-MC       XR          50058.0      1
M-MC       XY25        37700.0      1
L-LC       XY25        18862.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
